# Extraction of statistics from timeserie

In [2]:
import glob, os
import numpy as np
import pandas as pd
import rasterio
from rasterio import plot
from rasterio.plot import show
from pathlib import Path

## Set paths for input and output directories

In [3]:
#computer_path = 'X:/'
computer_path = '/Volumes/nbdid-sst-lbrat2104/'
grp_letter    = 'X'

# Directory for all work files
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'


# Directory where the images of the timeserie are stored
im_path = f'{work_path}NDVI/'

# Directory to store the statistics
stat_path = f'{work_path}NDVI_stat/'

Path(stat_path).mkdir(parents=True, exist_ok=True)


# Output files

ts_name = 'NDVI'

out_max = f'{stat_path}{ts_name}_mean_3_max.tif'
out_min = f'{stat_path}{ts_name}_mean_3_min.tif'


print(f'Input directory  : {im_path}')
print(f'Output directory : {stat_path}')

Input directory  : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/
Output directory : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI_stat/


## Check list of images

In [3]:
list_im = sorted(glob.glob(f'{im_path}*.tif'))

for im_file in list_im:
    print(im_file)

/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200116T105309_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200212T104049_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200316T104709_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200417T104021_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200520T105031_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200621T103629_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200719T105031_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200813T104629_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200914T104031_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20201019T103959_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20201118T104329_NDVI.tif
/Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20201218T104349_NDVI.tif


## Get metadata from one image of the list

In [4]:
src = rasterio.open(list_im[0], 'r')

profile = src.profile
dtype  = profile['dtype']
width  = profile['width']
height = profile['height']

print(f'Type   : {dtype}')
print(f'Width  : {width}')
print(f'Height : {height}')

src.close()

Type   : float64
Width  : 986
Height : 570


## Stack all raster into a single 3-D array

In [5]:
list_src_arr = []

for im_file in list_im:

    src = rasterio.open(im_file, "r")
    im = src.read(1)
    list_src_arr.append(im)
    src.close()


ts_arr = np.stack(list_src_arr, axis=0)

print(ts_arr.shape)


(12, 570, 986)


## Compute statistics on 3-D array

In [7]:
# Create empty array with NaN

result_arr_max = np.zeros((height, width)) * np.nan
result_arr_min = np.zeros((height, width)) * np.nan

# Remove values out of the range

max_val = 1
min_val = -1

ts_arr[(ts_arr >= max_val) | (ts_arr <= min_val)] = np.nan


# Compute the mean of the 3 max/min for every pixels
# --------------------------------------------------

for line in range(0,height):
    for col in range(0,width):
        
        # Get vector of timeserie values for a given pixel
        vec = ts_arr[:, line, col]

        # Sort vector
        sorted_vec = np.sort(vec)

        # Remove NaN from the sorted vector
        sorted_vec = sorted_vec[~np.isnan(sorted_vec)]
        
        if len(sorted_vec) > 3:
            
            # Compute mean of the 3 largest values
            result_arr_max[line, col] = sorted_vec[-3:].mean()
            
            # Compute mean of the 3 smallest values
            result_arr_min[line, col] = sorted_vec[:3].mean()
        
        else:
            # There are not enough values to calculate an average !
            result_arr_max[line, col] = np.nan
            result_arr_min[line, col] = np.nan


## Write statistics in new GeoTIFF file

In [8]:
dst = rasterio.open(out_max, "w", **profile)
dst.write(result_arr_max, 1)
dst.close()


dst = rasterio.open(out_min, "w", **profile)
dst.write(result_arr_min, 1)
dst.close()
